In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

In [4]:
symbol='AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 77252469,
 'calculationPrice': 'close',
 'change': -5.29,
 'changePercent': -0.03251,
 'close': 162.86,
 'closeSource': 'failocif',
 'closeTime': 1715704341440,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 161.12,
 'delayedPriceTime': 1653188518156,
 'extendedChange': 0.59,
 'extendedChangePercent': 0.00378,
 'extendedPrice': 157.5,
 'extendedPriceTime': 1679928952776,
 'high': 165.36,
 'highSource': 'aepimytel cd r5due ie1n',
 'highTime': 1658991885767,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 161.96,
 'iexCloseTime': 1660497587886,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 165.16,
 'iexOpenTime': 1701718560940,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'lastTradeTime': 1674971966500,
 'latestPrice': 158.16,
 'latestSource': 'Close',
 'latestTime': 'November 26, 2021',
 'latestUpdate': 1663012290093,
 'latestVolume': 80184075,
 'low': 

In [5]:
data['latestPrice']
data['marketCap']

2644280993437

In [6]:
my_columns = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy


In [7]:
inal_dataframe = final_dataframe.append(
                                        pd.Series(['AAPL', 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy


In [8]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)

In [9]:
final_dataframe



,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,157.98,47870941681,N/A
1,AAL,17.94,11664177480,N/A
2,AAP,237.83,14260338587,N/A
3,AAPL,162.97,2597209947971,N/A
4,ABBV,121.01,209224357902,N/A
...,...,...,...,...
500,YUM,127.80,37986072931,N/A
501,ZBH,127.41,26868680849,N/A
502,ZBRA,596.64,32204598884,N/A
503,ZION,64.40,10230049829,N/A


In [10]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [11]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,157.96,47789572707,N/A
1,AAL,18.41,11907719176,N/A
2,AAP,234.85,14705861456,N/A
3,AAPL,159.91,2637919041664,N/A
4,ABBV,119.45,207242750989,N/A
...,...,...,...,...
500,YUM,130.20,37672200032,N/A
501,ZBH,129.72,27121412036,N/A
502,ZBRA,590.83,31760854672,N/A
503,ZION,67.00,10185674875,N/A


In [12]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:1000000


In [13]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe



,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,157.96,47789572707,12
1,AAL,18.41,11907719176,107
2,AAP,234.85,14705861456,8
3,AAPL,159.91,2637919041664,12
4,ABBV,119.45,207242750989,16
...,...,...,...,...
500,YUM,130.20,37672200032,15
501,ZBH,129.72,27121412036,15
502,ZBRA,590.83,31760854672,3
503,ZION,67.00,10185674875,29


In [14]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)



In [15]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )



In [16]:
# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Price', string_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', string_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number Of Shares to Buy', string_format)
# writer.sheets['Recommended Trades'].set_column('A:A', 20, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 20, integer_format)

#This code works, but it violates the software principle of "Don't Repeat Yourself".

In [17]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)



In [18]:
writer.save()